## Bike Assignment By Jagadish 

## Pre-requisite - Load Data from CSV into Database

In [3]:
from sqlalchemy import create_engine
import pymysql
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [4]:
# Open database connection
conn = pymysql.connect(
        host='localhost',
        user='root', 
        password = "root",
        db='bike_sharing',
        )

cur = conn.cursor()

In [9]:
station = pd.read_csv("C:\\Users\\Jagad\\Downloads\\Bike Sharing Dataset-20220113T044816Z-001\\Bike Sharing Dataset\\station.csv")
status = pd.read_csv("C:\\Users\\Jagad\\Downloads\\Bike Sharing Dataset-20220113T044816Z-001\\Bike Sharing Dataset\\status.csv")
trip = pd.read_csv("C:\\Users\\Jagad\\Downloads\\Bike Sharing Dataset-20220113T044816Z-001\\Bike Sharing Dataset\\trip.csv")
#weather = pd.read_csv("C:\\Users\\Jagad\\Downloads\\Bike Sharing Dataset-20220113T044816Z-001\\Bike Sharing Dataset\\weather.csv")
weather = pd.read_csv("C:\\Users\\Jagad\MS Program\\weather.csv")


In [14]:
# overwriting data after changing format
station["installation_date"]= pd.to_datetime(station["installation_date"])
station.to_csv('station_updated.csv', index=False)
status["time"]= pd.to_datetime(status["time"])
status.to_csv('status_updated.csv', index=False)
trip["start_date"]= pd.to_datetime(trip["start_date"])
trip["end_date"]= pd.to_datetime(trip["end_date"])
trip.to_csv('trip_updated.csv', index=False)
weather["date"]= pd.to_datetime(weather["date"])
weather.to_csv('weather_updated.csv', index=False)

## Loaded the 4 files into the database 

mysql> load data infile 'C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/station_updated.csv' into table station fields terminated by ',' enclosed by '"' lines terminated by '\n' ignore 1 rows;
Query OK, 70 rows affected (0.04 sec)
Records: 70  Deleted: 0  Skipped: 0  Warnings: 0

mysql> load data infile 'C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/status_updated.csv' into table status fields terminated by ',' enclosed by '"' lines terminated by '\n' ignore 1 rows;
Query OK, 71984434 rows affected (21 min 55.21 sec)
Records: 71984434  Deleted: 0  Skipped: 0  Warnings: 0

mysql> load data infile 'C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/trip_updated.csv' into table trip fields terminated by ',' enclosed by '"' lines terminated by '\n' ignore 1 rows;
Query OK, 669959 rows affected (19.11 sec)
Records: 669959  Deleted: 0  Skipped: 0  Warnings: 0

mysql> load data infile 'C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/weather_updated.csv' into table weather fields terminated by ',' enclosed by '"' lines terminated by '\n' ignore 1 rows;
Query OK, 3665 rows affected (0.17 sec)
Records: 3665  Deleted: 0  Skipped: 0  Warnings: 0

## Task 1: Get to Know Your Company

## Question 1.1 

What are the total numbers of:

Bike stations?

Bikes?

Trips?

In [38]:
# Total Number of Bike Stations
cur.execute("SELECT count(*) FROM station")
NumBikeStations = cur.fetchone()
print("Number of Bike Stations is", NumBikeStations[0])

cur.execute("select count(distinct bike_id) from trip")
Bikecount = cur.fetchone()
print("Number of Bikes is", Bikecount[0])

cur.execute("select count(*) as TripsCount from trip")
Tripcount = cur.fetchone()
print("Number of Bikes is", Tripcount[0])



Number of Bike Stations is 70
Number of Bikes is 700
Number of Bikes is 669959


## Question 1.3 

What is the relationship between the following columns (one to one, many to one, many to many)?

bike_id (Trip table) and start_station_id (Trip table)

pincode (Weather table) and station location (latitude and longitude in Station table)

8/29/2013 (date column in Weather table) and mean wind speed (Weather table)

## Answer 1.3.1

1.  bike_id (Trip table) and start_station_id (Trip table) has many to many relationship
2. The below two sql shows that one bike can be in many stations and 1 station can have many bikes

In [59]:
# pd.read_sql("select bike_id, start_station_id from trip  where bike_id=9 order by bike_id limit 5", conn)
cur.execute("select bike_id, start_station_id from trip  where bike_id=9 order by bike_id limit 5")
for row in cur.fetchall():
    print(row)

(9, 37)
(9, 35)
(9, 35)
(9, 35)
(9, 36)


In [61]:
# pd.read_sql("select start_station_id, bike_id from trip where start_station_id=37 order by start_station_id limit 5", conn)
cur.execute("select start_station_id, bike_id from trip where start_station_id=37 order by start_station_id limit 5")
for row in cur.fetchall():
    print(row)

(37, 9)
(37, 95)
(37, 20)
(37, 15)
(37, 100)


## Answer 1.3.2

1. There is no relationship between zip code in weather table and latitude and longitude in stations table.
However logically one zipcode can have multiple longitude and latitude.

In [62]:
# pd.read_sql("select * from weather", conn)
cur.execute("select * from weather limit 5")
for row in cur.fetchall():
    print(row)

(0, datetime.datetime(2013, 8, 29, 0, 0), 68.0, 75.0, 58.0, 11.0, '94107\r')
(1, datetime.datetime(2013, 8, 30, 0, 0), 69.0, 70.0, 58.0, 13.0, '94107\r')
(2, datetime.datetime(2013, 8, 31, 0, 0), 64.0, 75.0, 56.0, 15.0, '94107\r')
(3, datetime.datetime(2013, 9, 1, 0, 0), 66.0, 68.0, 56.0, 13.0, '94107\r')
(4, datetime.datetime(2013, 9, 2, 0, 0), 69.0, 77.0, 60.0, 12.0, '94107\r')


In [63]:
# pd.read_sql("select * from weather", conn)
cur.execute("select * from station limit 5")
for row in cur.fetchall():
    print(row)

(2, 'San Jose Diridon Caltrain Station', 37.3297, -121.902, 27, 'San Jose', datetime.datetime(2013, 8, 6, 0, 0))
(3, 'San Jose Civic Center', 37.3307, -121.889, 15, 'San Jose', datetime.datetime(2013, 8, 5, 0, 0))
(4, 'Santa Clara at Almaden', 37.334, -121.895, 11, 'San Jose', datetime.datetime(2013, 8, 6, 0, 0))
(5, 'Adobe on Almaden', 37.3314, -121.893, 19, 'San Jose', datetime.datetime(2013, 8, 5, 0, 0))
(6, 'San Pedro Square', 37.3367, -121.894, 15, 'San Jose', datetime.datetime(2013, 8, 7, 0, 0))


## Answer 1.3.3

1. The below 2 SQL shows that date and windspeed has many to many relationship

In [67]:
# pd.read_sql("select * from weather where date='8/29/2013' order by date limit 5", conn)
# pd.read_sql("select * from weather where date='8/29/2013' order by mean_wind_speed_mph limit 5", conn)

cur.execute("select date, mean_wind_speed_mph from weather order by date limit 5")
for row in cur.fetchall():
    print(row)

(datetime.datetime(2013, 8, 29, 0, 0), 11.0)
(datetime.datetime(2013, 8, 29, 0, 0), 5.0)
(datetime.datetime(2013, 8, 29, 0, 0), 8.0)
(datetime.datetime(2013, 8, 29, 0, 0), 7.0)
(datetime.datetime(2013, 8, 29, 0, 0), 6.0)


## Answer 1.4

In [42]:
# pd.read_sql("select id as FirstTripID from trip order by start_date asc limit 1", conn)
# pd.read_sql("select id as LastTripID from trip order by end_date desc limit 1", conn)

cur.execute("select * from trip order by start_date asc limit 1")
firstTrip = cur.fetchone()
print("************************************************************************************************************")  
print("First Trip in Trip Table is ", firstTrip)

cur.execute("select * from trip order by start_date desc limit 1")
lastTrip = cur.fetchone()
print("************************************************************************************************************")  
print("Last Trip in Trip Table is ", lastTrip)
print("************************************************************************************************************")  


************************************************************************************************************
First Trip in Trip Table is  (4069, 174, datetime.datetime(2013, 8, 29, 9, 8), '2nd at South Park', 64, datetime.datetime(2013, 8, 29, 9, 11), '2nd at South Park', 64, 288, 'Subscriber', '94114\r')
************************************************************************************************************
Last Trip in Trip Table is  (913460, 765, datetime.datetime(2015, 8, 31, 23, 26), 'Harry Bridges Plaza (Ferry Building)', 50, datetime.datetime(2015, 8, 31, 23, 39), 'San Francisco Caltrain (Townsend at 4th)', 70, 288, 'Subscriber', '2139\r')
************************************************************************************************************


## Answer 1.5

In [44]:
# pd.read_sql("select avg(duration) as AverageTripDuration from trip", conn)
# pd.read_sql("select avg(duration) as AverageTripDuration from trip where start_station_name=end_station_name", conn)

cur.execute("select avg(duration) as AverageTripDuration from trip")
AvgAllTrips = cur.fetchone()
print("************************************************************************************************************")  
print("Average Duration of all Trips is : ", AvgAllTrips[0])

cur.execute("select avg(duration) as AverageTripDuration from trip where start_station_name=end_station_name")
AvgStartFinishSame = cur.fetchone()
print("************************************************************************************************************")  
print("Average duration of trips where start and end station is same : ", AvgStartFinishSame[0])
print("************************************************************************************************************")  

************************************************************************************************************
Average Duration of all Trips is :  1107.9498
************************************************************************************************************
Average duration of trips where start and end station is same :  6357.4011
************************************************************************************************************


## Answer 1.6

In [45]:
# pd.read_sql("select bike_id, sum(duration) from trip group by bike_id order by sum(duration) desc limit 3", conn)
cur.execute("select bike_id, sum(duration) from trip group by bike_id order by sum(duration) desc limit 1")
MostUsed = cur.fetchone()
print("Most used bike in terms of duration (with BIKE ID is ) : ", MostUsed[0])

Most used bike in terms of duration (with BIKE ID is ) :  535


In [57]:
## Answers for 1.2 and 1.7 is in Tableau 

## Task 2 - Demand Prediction

## Answer 2.1

In [80]:
#What are the top 10 least popular stations? Hint: Find the least frequently appearing start stations from the Trip table

In [82]:
# pd.read_sql("select start_station_name, count(*) from trip group by start_station_name order by count(*) asc limit 10", conn)
cur.execute("select start_station_name, count(*) from trip group by start_station_name order by count(*) asc limit 10")
print("Top 10 least popular stations are : ")
for row in cur.fetchall():
    #print("************************************************************************************************************")  
    print(row[0])


Top 10 least popular stations are : 
San Jose Government Center
Broadway at Main
Redwood City Public Library
Franklin at Maple
San Mateo County Center
Redwood City Medical Center
Mezes Park
Stanford in Redwood City
Park at Olive
Santa Clara County Civic Center


## Answer 2.2

In [83]:
# Find the idle time for Station 2 on 29 July, 2018

In [56]:
# select bikes_available, time  from status where station_id=2 and bikes_available>3 and time >='2018-07-29 00:00:00' and time <= '2018-07-30 00:00:00'
cur.execute("select count(*) from status where station_id=2 and bikes_available>3 and time >='2018-07-29 00:00:00' and time <= '2018-07-30 00:00:00'")
NumofIdleBikes = cur.fetchone()
print("Number of idle bikes greater than 3 in station 2 for July-29-2018 is ", NumofIdleBikes[0])
cur.execute("select bikes_available, time  from status where station_id=2 and bikes_available>3 and time >='2018-07-29 00:00:00' and time <= '2018-07-30 00:00:00'")
for row in cur.fetchall():
    #print("************************************************************************************************************")  
    print(row)

Number of idle bikes greater than 3 in station 2 for July-29-2018 is  0


Answer 2.2 : The data set does not contain data for July 29 2018

## Answer 2.3 

In [1]:
# Find the distance between consecutive stations (between Stations 1 and 2, Stations 2 and 3, and so on).

In [5]:
import sqlite3
sqlliteconn = sqlite3.connect(database="bikeassignment.db")

In [6]:
stationdf = pd.read_sql("select station_name, latitude, longitude from station", conn)
stations_list = stationdf.values.tolist()

In [7]:
from math import cos, asin, sqrt, pi

def distance(lat1, lon1, lat2, lon2):
    p = pi/180
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p) * cos(lat2*p) * (1-cos((lon2-lon1)*p))/2
    return 12742 * asin(sqrt(a)) #2*R*asin...

In [8]:
DistanceinKMList = []

for i in range(0, len(stations_list)-1):
    lat1, lon1 = stations_list[i][1], stations_list[i][2]
    lat2, lon2 = stations_list[i+1][1], stations_list[i+1][2]
    distanceinKM = distance(lat1, lon1, lat2, lon2)
    #print("Distance in Kilo meters in ", distanceinKM)
    DistanceinKMList.append([stations_list[i][0], stations_list[i+1][0], distanceinKM ])

In [10]:
Distance = pd.DataFrame(DistanceinKMList, columns = ['StationN1', 'StationN2', 'DistanceinKM'])

In [11]:
#Distance
Distance.to_sql("Distance",sqlliteconn, if_exists="replace", index=False )

In [12]:
#Distance between two consecutive stations is provided in ascending order
pd.read_sql("select * from Distance  order by DistanceinKM asc limit 20", sqlliteconn)

,StationN1,StationN2,DistanceinKM
0,San Francisco Caltrain 2 (330 Townsend),San Francisco Caltrain (Townsend at 4th),0.000000
1,Palo Alto Caltrain Station,University and Emerson,0.185112
2,San Jose City Hall,MLK Library,0.188775
3,Commercial at Montgomery,Washington at Kearney,0.220652
4,Redwood City Caltrain Station,San Mateo County Center,0.242816
5,2nd at Folsom,Howard at 2nd,0.258114
6,South Van Ness at Market,Market at 10th,0.266381
7,Clay at Battery,Davis at Jackson,0.310304
8,San Francisco City Hall,Golden Gate at Polk,0.312825
9,Temporary Transbay Terminal (Howard at Beale),Beale at Market,0.328881


	StationN1	StationN2	DistanceinKM
0	San Francisco Caltrain 2 (330 Townsend)	San Francisco Caltrain (Townsend at 4th)	0.000000

1	Palo Alto Caltrain Station	University and Emerson	0.185112

2	San Jose City Hall	MLK Library	0.188775

3	Commercial at Montgomery	Washington at Kearney	0.220652

4	Redwood City Caltrain Station	San Mateo County Center	0.242816

5	2nd at Folsom	Howard at 2nd	0.258114

6	South Van Ness at Market	Market at 10th	0.266381

7	Clay at Battery	Davis at Jackson	0.310304

8	San Francisco City Hall	Golden Gate at Polk	0.312825

9	Temporary Transbay Terminal (Howard at Beale)	Beale at Market	0.328881

10	Santa Clara at Almaden	Adobe on Almaden	0.338897

11	MLK Library	SJSU 4th at San Carlos	0.387412

12	Paseo de San Antonio	San Salvador at 1st	0.409950

13	San Mateo County Center	Redwood City Public Library	0.461517

14	Harry Bridges Plaza (Ferry Building)	Embarcadero at Folsom	0.507494

15	Market at 4th	Market at Sansome	0.508135

16	Steuart at Market	Mechanics Plaza (Market at Battery)	0.538483

17	Embarcadero at Folsom	Embarcadero at Bryant	0.545997

18	Davis at Jackson	Commercial at Montgomery	0.558420

19	Adobe on Almaden	San Pedro Square	0.595928

## Answer 2.4

In [27]:
# Find the least popular stations
pd.read_sql("select start_station_name, count(*) as Popularity from trip group by start_station_name order by Popularity asc", conn)

,start_station_name,Popularity
0,San Jose Government Center,23
1,Broadway at Main,67
2,Redwood City Public Library,213
3,Franklin at Maple,224
4,San Mateo County Center,287
...,...,...
69,Temporary Transbay Terminal (Howard at Beale),26089
70,Embarcadero at Sansome,27713
71,Harry Bridges Plaza (Ferry Building),32934
72,San Francisco Caltrain 2 (330 Townsend),33742


In [58]:
# Find the distance between the top 4 least popular stations 
pd.read_sql("select * from Distance  where StationN1 in ('San Jose Government Center', 'Broadway at Main', 'Redwood City Public Library', 'Franklin at Maple') ", sqlliteconn)

,StationN1,StationN2,DistanceinKM
0,Franklin at Maple,Redwood City Caltrain Station,0.650581
1,Redwood City Public Library,Stanford in Redwood City,2.121835


In [59]:
# Find the distance between the top 4 least popular stations 
pd.read_sql("select * from Distance  where StationN2 in ('San Jose Government Center', 'Broadway at Main', 'Redwood City Public Library', 'Franklin at Maple') ", sqlliteconn)

,StationN1,StationN2,DistanceinKM
0,SJSU - San Salvador at 9th,Franklin at Maple,35.010941
1,San Mateo County Center,Redwood City Public Library,0.461517


## Answer -  I would close the follwing 3 stations - 
#1. "Franklin at Maple" can be closed as it is less popular than Redwood city and these two stations are nearby
#2. "Redwood City Public Library" can be closed as it is less popular than "Standford in Redwood City "and they are close by 
#3. "San Mateo county center" can be closed as it is less popular and it is close to "Stanford in Redwood City"

## Task 3 - Optimizing Operations

## Answer 3.1
Calculate the average number of bikes and docks available for Station 2 and Station 3 (Hint: Use the Status table.)

In [37]:
#Station2 = pd.read_sql("select avg(bikes_available), avg(docks_available) from status where station_id=2", conn)
#Station3 = pd.read_sql("select avg(bikes_available), avg(docks_available) from status where station_id=3", conn)

cur.execute("select avg(bikes_available), avg(docks_available) from status where station_id=2")
Station2 = cur.fetchone()
print("For Station 2 : Average Bikes available is : ", Station2[0])
print("For Station 2 : Average Docks available is : ", Station2[1])

cur.execute("select avg(bikes_available), avg(docks_available) from status where station_id=3")
Station3 = cur.fetchone()
print("For Station 3 : Average Bikes available is : ", Station3[0])
print("For Station 3 : Average Docks available is : ", Station3[1])


For Station 2 : Average Bikes available is :  13.1726
For Station 2 : Average Docks available is :  13.7615
For Station 3 : Average Bikes available is :  8.4611
For Station 3 : Average Docks available is :  6.5279


## Answers for 3.2 and 3.3 is in Tableau


## Answer 3.4

North of San Francisco is the most popular station (shown with bigger circles in tableau in 3.2). 

The peak business hours is from 7 AM to 9 AM and between 4 PM to 6 PM (shown in Tree graph in Tableau). 

Recommendations is to keep more bikes available in the popular locations between the peak business hours. 


## Task 4: Couple Bikes

In [60]:
# Zulip has decided to start a new product line called Couple Bikes.
# This will enable two persons to travel from one station to another at the same time. 
# What are some of the factors that you will have to consider while validating the idea of couple bikes?

In [64]:
# Find the Top 10 popular stations
cur.execute("select start_station_name, count(*) from trip group by start_station_name order by count(*) desc limit 10")
print("Top 10 least popular stations are : ")
for row in cur.fetchall():
    #print("************************************************************************************************************")  
    print(row[0], row[1])


Top 10 least popular stations are : 
San Francisco Caltrain (Townsend at 4th) 49092
San Francisco Caltrain 2 (330 Townsend) 33742
Harry Bridges Plaza (Ferry Building) 32934
Embarcadero at Sansome 27713
Temporary Transbay Terminal (Howard at Beale) 26089
2nd at Townsend 25837
Steuart at Market 24838
Market at Sansome 24172
Townsend at 7th 23724
Market at 10th 20272


In [65]:
# Identify the stations where bikes are returned to the same place
cur.execute("select start_station_name, count(*) from trip where start_station_name=end_station_name group by start_station_name order by count(*) desc limit 10")
print("Top 10 least popular stations are where bikes are returned to same station: ")
for row in cur.fetchall():
    #print("************************************************************************************************************")  
    print(row)


Top 10 least popular stations are where bikes are returned to same station: 
('Embarcadero at Sansome', 2092)
('Harry Bridges Plaza (Ferry Building)', 1733)
('University and Emerson', 894)
('Market at 4th', 724)
('2nd at Townsend', 683)
('Steuart at Market', 668)
('Market at 10th', 666)
('Embarcadero at Vallejo', 620)
('Powell at Post (Union Square)', 619)
('Powell Street BART', 614)


## Answer Task 4 

Recommendation is to do a Trial for couple bikes in TOP 4 popular stations.

The first four stations are very popular stations. So more users could be benefited.
Also the last two stations are popular as well as bikes are returned to the same location.

1. San Francisco Caltrain (Townsend at 4th) 

2. San Francisco Caltrain 2 (330 Townsend) 

3. Harry Bridges Plaza (Ferry Building) 

4. Embarcadero at Sansome 



## Answer for Task 5 is in PPT - Bike Assignment Jagadish Janakiraman v1.0 01172022.pptx